In [1]:
!pip install transformers tensorflow scikit-learn pandas

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

In [3]:
# Load Dataset
df = pd.read_csv('/content/drive/MyDrive/Smart Study Assistant Project/Model for Gate questions classification/questions-data-new.csv')  # Use your actual file path

text_column = 'question'
label_column = 'topic'

# Drop NaNs just in case
df = df[[text_column, label_column]].dropna()

In [4]:
# Label Encoding
le = LabelEncoder()
df['label'] = le.fit_transform(df[label_column])

In [5]:
# Split into Train / Val / Test (70/15/15)
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df['question'].tolist(), df['label'].tolist(), test_size=0.3, random_state=42, stratify=df['label']
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

In [6]:
# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
max_len = 128

def tokenize(texts):
    return tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='tf'
    )

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)
test_encodings = tokenize(test_texts)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
# Create TensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(1000).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(64)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(64)

In [8]:
# Load and Compile Model
num_labels = len(le.classes_)
model = TFDistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=num_labels
)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [9]:
# Train the Model
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)

Epoch 1/3
105/105 [==============================] - 69s 227ms/step - loss: 1.5399 - accuracy: 0.5679 - val_loss: 0.8694 - val_accuracy: 0.8167
Epoch 2/3
105/105 [==============================] - 23s 220ms/step - loss: 0.5897 - accuracy: 0.8815 - val_loss: 0.4472 - val_accuracy: 0.8722
Epoch 3/3
105/105 [==============================] - 24s 227ms/step - loss: 0.2810 - accuracy: 0.9405 - val_loss: 0.3736 - val_accuracy: 0.8917


In [10]:
# Evaluate on Test Set
test_logits = model.predict(test_dataset).logits
test_preds = np.argmax(test_logits, axis=1)

acc = accuracy_score(test_labels, test_preds)
f1 = f1_score(test_labels, test_preds, average='weighted')

print(f"\n Test Accuracy: {acc:.4f}")
print(f" Test F1 Score: {f1:.4f}")

6/6 [==============================] - 4s 236ms/step

 Test Accuracy: 0.9111
 Test F1 Score: 0.9117


In [11]:
# Inference
def predict(texts):
    # Tokenize the texts as a batch
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors='tf',
        max_length=128
    )

    # Predict using the model
    outputs = model(inputs)
    logits = outputs.logits

    # Get predicted class indices
    preds = tf.argmax(logits, axis=1).numpy()

    # Decode predicted class labels
    labels = le.inverse_transform(preds)

    return labels

In [12]:
print(predict("Consider a demand paging memory management system with 32-bit logical address, 20-bit physical address, and page size of 2048 bytes. Assuming that the memory is byte addressable, what is the maximum number of entries in the page table?"))

['Operating System']


In [13]:
import os
import pickle
save_dir = '/content/drive/MyDrive/Smart Study Assistant Project/Model for Gate questions classification/main_model'
os.makedirs(save_dir, exist_ok=True)

# Save model and tokenizer
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# Save label encoder
with open(os.path.join(save_dir, 'label_encoder.pkl'), 'wb') as f:
    pickle.dump(le, f)

In [14]:
text = [ "Ravi had ______ younger brother who taught at ______ university. He was widely regarded as ______ honorable man. Select the option with the correct sequence of articles to fill in the blanks.", "The CEO’s decision to downsize the workforce was considered myopic because it sacrificed long-term stability to accommodate short-term gains. Select the most appropriate option that can replace the word “myopic” without changing the meaning of the sentence.", "The average marks obtained by a class in an examination were calculated as 30.8. However, while checking the marks entered, the teacher found that the marks of one student were entered incorrectly as 24 instead of 42. After correcting the marks, the average becomes 31.4. How many students does the class have?", "Consider the relationships among P, Q, R, S, and T: • P is the brother of Q. • S is the daughter of Q. • T is the sister of S. • R is the mother of Q. The following statements are made based on the relationships given above. (1) R is the grandmother of S. (2) P is the uncle of S and T. (3) R has only one son. (4) Q has only one daughter. Which one of the following options is correct?", "According to the map shown in the figure, which one of the following statements is correct? Note: The figure shown is representative.", "“I put the brown paper in my pocket along with the chalks, and possibly other things. I suppose every one must have reflected how primeval and how poetical are the things that one carries in one’s pocket: the pocket-knife, for instance the type of all human tools, the infant of the sword. Once I planned to write a book of poems entirely about the things in my pocket. But I found it would be too long: and the age of the great epics is past.” (From G.K. Chesterton’s “A Piece of Chalk”) Based only on the information provided in the above passage, which one of the following statements is true?", "In the diagram, the lines QR and ST are parallel to each other. The shortest distance between these two lines is half the shortest distance between the point P and line QR. What is the ratio of the area of the triangle PST to the area of the trapezium SQRT? Note: The figure shown is representative.", "A fair six-faced dice, with the faces labelled ‘1’, ‘2’, ‘3’, ‘4’, ‘5’, and ‘6’, is rolled thrice. What is the probability of rolling ‘6’ exactly once?", "A square paper, shown in figure (I), is folded along the dotted lines as shown in the figures (II) and (III). Then a few cuts are made as shown in figure (IV). Which one of the following patterns will be obtained when the paper is unfolded? Note: The figures shown are representative.", "A shop has 4 distinct flavors of ice-cream. One can purchase any number of scoops of any flavor. The order in which the scoops are purchased is inconsequential. If one wants to purchase 3 scoops of ice-cream, in how many ways can one make that purchase?", "Suppose a program is running on a non-pipelined single processor computer system. The computer is connected to an external device that can interrupt the processor asynchronously. The processor needs to execute the interrupt service routine (ISR) to serve this interrupt. The following steps (not necessarily in order) are taken by the processor when the interrupt arrives: (i) The processor saves the content of the program counter. (ii) The program counter is loaded with the start address of the ISR. (iii) The processor finishes the present instruction. Which ONE of the following is the CORRECT sequence of steps?", "Which ONE of the following statements is FALSE regarding the symbol table?", "Which ONE of the following techniques used in compiler code optimization uses live variable analysis?", "Consider a demand paging memory management system with 32-bit logical address, 20-bit physical address, and page size of 2048 bytes. Assuming that the memory is byte addressable, what is the maximum number of entries in the page table?", "A schedule of three database transactions 𝑇1, 𝑇2, and 𝑇3 is shown. 𝑅𝑖(𝐴) and 𝑊𝑖(𝐴) denote read and write of data item 𝐴 by transaction 𝑇𝑖, 𝑖= 1,2,3. The transaction 𝑇1 aborts at the end. Which other transaction(s) will be required to be rolled back? 𝑅1(𝑋) 𝑊1(𝑌) 𝑅2(𝑋) 𝑅2(𝑌) 𝑅3(𝑌) 𝐴𝐵𝑂𝑅𝑇(𝑇1)", "Identify the ONE CORRECT matching between the OSI layers and their corresponding functionalities as shown.", "𝑔(. ) is a function from 𝐴 to 𝐵, 𝑓(. ) is a function from 𝐵 to 𝐶, and their composition defined as 𝑓(𝑔(. )) is a mapping from 𝐴 to 𝐶. If 𝑓(. ) and 𝑓(𝑔(. )) are onto (surjective) functions, which ONE of the following is TRUE about the function 𝑔(. )?", "Let 𝐺 be any undirected graph with positive edge weights, and 𝑇 be a minimum spanning tree of 𝐺. For any two vertices, 𝑢 and 𝑣, let 𝑑1(𝑢, 𝑣) and 𝑑2(𝑢, 𝑣) be the shortest distances between 𝑢 and 𝑣 in 𝐺 and 𝑇, respectively. Which ONE of the options is CORRECT for all possible 𝐺, 𝑇, 𝑢 and 𝑣?", "Consider the following context-free grammar 𝐺, where 𝑆, 𝐴, and 𝐵 are the variables (non-terminals), 𝑎 and 𝑏 are the terminal symbols, 𝑆 is the start variable, and the rules of 𝐺 are described as: 𝑆→𝑎𝑎𝐵 | 𝐴𝑏𝑏 𝐴→𝑎 | 𝑎𝐴 𝐵→𝑏 | 𝑏𝐵 Which ONE of the languages 𝐿(𝐺) is accepted by 𝐺?", "Consider the following recurrence relation: 𝑇(𝑛) = 2𝑇(𝑛−1) + 𝑛2𝑛 for 𝑛 > 0, 𝑇(0) = 1. Which ONE of the following options is CORRECT?", "Consider the following 𝐵+ tree with 5 nodes, in which a node can store at most 3 key values. The value 23 is now inserted in the 𝐵+ tree. Which of the following options(s) is/are CORRECT?", "Consider the 3-way handshaking protocol for TCP connection establishment. Let the three packets exchanged during the connection establishment be denoted as P1, P2, and P3, in order. Which of the following option(s) is/are TRUE with respect to TCP header flags that are set in the packets?", "Consider the given system of linear equations for variables 𝑥 and 𝑦, where 𝑘 is a real-valued constant. Which of the following option(s) is/are CORRECT? 𝑥+ 𝑘𝑦= 1𝑘𝑥+ 𝑦= −1", "Let 𝑋 be a 3-variable Boolean function that produces output as ‘1’ when at least two of the input variables are ‘1’. Which of the following statement(s) is/are CORRECT, where 𝑎, 𝑏, 𝑐, 𝑑, 𝑒 are Boolean variables?", "The number −6 can be represented as 1010 in 4-bit 2’s complement representation. Which of the following is/are CORRECT 2’s complement representation(s) of −6?", "Which of the following statement(s) is/are TRUE for any binary search tree (BST) having 𝑛 distinct integers?", "A partial data path of a processor is given in the figure, where RA, RB, and RZ are 32-bit registers. Which option(s) is/are CORRECT related to arithmetic operations using the data path as shown?", "A regular language 𝐿 is accepted by a non-deterministic finite automaton (NFA) with 𝑛 states. Which of the following statement(s) is/are FALSE?", "Suppose in a multiprogramming environment, the following C program segment is executed. A process goes into I/O queue whenever an I/O related operation is performed. Assume that there will always be a context switch whenever a process requests for an I/O, and also whenever the process returns from an I/O. The number of times the process will enter the ready queue during its lifetime (not counting the time the process enters the ready queue when it is run initially) is _______. (Answer in integer)", "Let 𝑆 be the set of all ternary strings defined over the alphabet {𝑎, 𝑏, 𝑐}. Consider all strings in 𝑆 that contain at least one occurrence of two consecutive symbols, that is, “aa”, “bb” or “cc”. The number of such strings of length 5 that are possible is _______. (Answer in integer)", "Consider the given function 𝑓(𝑥). 𝑓(𝑥) = { 𝑎𝑥+ 𝑏 for 𝑥< 1𝑥3 + 𝑥2 + 1 for 𝑥≥1 If the function is differentiable everywhere, the value of 𝑏 must be ________. (rounded off to one decimal place)", "A box contains 5 coins: 4 regular coins and 1 fake coin. When a regular coin is tossed, the probability 𝑃(ℎ𝑒𝑎𝑑) = 0.5 and for a fake coin, 𝑃(ℎ𝑒𝑎𝑑) = 1. You pick a coin at random and toss it twice, and get two heads. The probability that the coin you have chosen is the fake coin is _______. (rounded off to two decimal places)", "The pseudocode of a function fun() is given below: fun(int A[0,…,n-1]){ for i=0 to n-2 for j=0 to n-i-2 if (A[j]>A[j+1]) then swap A[j] and A[j+1] } Let 𝐴[0, … ,29] be an array storing 30 distinct integers in descending order. The number of swap operations that will be performed, if the function fun() is called with 𝐴[0, … ,29] as argument, is __________. (Answer in integer)", "The output of the given C program is __________. (Answer in integer)", "The height of any rooted tree is defined as the maximum number of edges in the path from the root node to any leaf node. Suppose a Min-Heap 𝑇 stores 32 keys. The height of 𝑇 is _____________. (Answer in integer)", "Consider a memory system with 1M bytes of main memory and 16K bytes of cache memory. Assume that the processor generates 20-bit memory address, and the cache block size is 16 bytes. If the cache uses direct mapping, how many bits will be required to store all the tag values? [Assume memory is byte addressable, 1K=210, 1M=220 .]", "A processor has 64 general-purpose registers and 50 distinct instruction types. An instruction is encoded in 32-bits. What is the maximum number of bits that can be used to store the immediate operand for the given instruction? ADD R1, #25 // R1 = R1 + 25", "A computer has two processors, 𝑀1 and 𝑀2. Four processes 𝑃1, 𝑃2, 𝑃3, 𝑃4 with CPU bursts of 20, 16, 25, and 10 milliseconds, respectively, arrive at the same time and these are the only processes in the system. The scheduler uses non-preemptive priority scheduling, with priorities decided as follows: • 𝑀1 uses priority of execution for the processes as, 𝑃1 > 𝑃3 > 𝑃2 > 𝑃4, i.e., 𝑃1 and 𝑃4 have highest and lowest priorities, respectively. • 𝑀2 uses priority of execution for the processes as, 𝑃2 > 𝑃3 > 𝑃4 > 𝑃1, i.e., 𝑃2 and 𝑃1 have highest and lowest priorities, respectively. A process 𝑃𝑖 is scheduled to a processor 𝑀𝑘, if the processor is free and no other process 𝑃𝑗 is waiting with higher priority. At any given point of time, a process can be allocated to any one of the free processors without violating the execution priority rules. Ignore the context switch time. What will be the average waiting time of the processes in milliseconds?", "Consider two relations describing 𝑡𝑒𝑎𝑚𝑠 and 𝑝𝑙𝑎𝑦𝑒𝑟𝑠 in a sports league: • 𝑡𝑒𝑎𝑚𝑠(𝑡𝑖𝑑, 𝑡𝑛𝑎𝑚𝑒): 𝑡𝑖𝑑, 𝑡𝑛𝑎𝑚e are team-id and team-name, respectively • 𝑝𝑙𝑎𝑦𝑒𝑟𝑠(𝑝𝑖𝑑, 𝑝𝑛𝑎𝑚𝑒, 𝑡𝑖𝑑): 𝑝𝑖𝑑, 𝑝𝑛𝑎𝑚𝑒, and 𝑡𝑖𝑑 denote player-id, player-name and the team-id of the player, respectively Which ONE of the following tuple relational calculus queries returns the name of the players who play for the team having 𝑡𝑛𝑎𝑚𝑒 as ′𝑀𝐼′?", "A packet with the destination IP address 145.36.109.70 arrives at a router whose routing table is shown. Which interface will the packet be forwarded to?", "Let 𝐴 be a 2 × 2 matrix as given. 𝐴= [ 111−1 ] What are the eigenvalues of the matrix 𝐴13 ?", "Consider the following four variable Boolean function in sum-of-product form 𝐹(𝑏3, 𝑏2, 𝑏1, 𝑏0) = ∑(0, 2, 4, 8, 10, 11, 12). where the value of the function is computed by considering 𝑏3𝑏2𝑏1𝑏0 as a 4-bit binary number, where 𝑏3 denotes the most significant bit and 𝑏0 denotes the least significant bit. Note that there are no don’t care terms. Which ONE of the following options is the CORRECT minimized Boolean expression for 𝐹?", "Let 𝐺(𝑉, 𝐸) be an undirected and unweighted graph with 100 vertices. Let 𝑑(𝑢, 𝑣) denote the number of edges in a shortest path between vertices 𝑢 and 𝑣 in 𝑉. Let the maximum value of 𝑑(𝑢, 𝑣), 𝑢, 𝑣∈𝑉 such that 𝑢≠𝑣, be 30. Let 𝑇 be any breadth-first-search tree of 𝐺. Which ONE of the given options is CORRECT for every such graph 𝐺?", "Consider the following two languages over the alphabet {𝑎, 𝑏}: 𝐿1 = { 𝛼𝛽𝛼 | 𝛼∈{𝑎, 𝑏}+ AND 𝛽∈{𝑎, 𝑏}+ } 𝐿2 = { 𝛼𝛽𝛼 | 𝛼∈{𝑎}+ AND 𝛽∈{𝑎, 𝑏}+ } Which ONE of the following statements is CORRECT?", "Consider the following two languages over the alphabet {𝑎, 𝑏, 𝑐}, where 𝑚 and 𝑛 are natural numbers. 𝐿1 = {𝑎𝑚𝑏𝑚𝑐𝑚+𝑛 | 𝑚, 𝑛≥1} 𝐿2 = {𝑎𝑚𝑏𝑛𝑐𝑚+𝑛 | 𝑚, 𝑛≥1} Which ONE of the following statements is CORRECT?", "Which of the following statement(s) is/are TRUE while computing First and Follow during top down parsing by a compiler?", "Consider a relational schema 𝑡𝑒𝑎𝑚(𝑛𝑎𝑚𝑒, 𝑐𝑖𝑡𝑦, 𝑜𝑤𝑛𝑒𝑟), with functional dependencies {𝑛𝑎𝑚𝑒→𝑐𝑖𝑡𝑦, 𝑛𝑎𝑚𝑒→𝑜𝑤𝑛𝑒𝑟}. The relation 𝑡𝑒𝑎𝑚 is decomposed into two relations, 𝑡1(𝑛𝑎𝑚𝑒, 𝑐𝑖𝑡𝑦) and 𝑡2(𝑛𝑎𝑚𝑒, 𝑜𝑤𝑛𝑒𝑟). Which of the following statement(s) is/are TRUE?", "Which of the following predicate logic formulae/formula is/are CORRECT representation(s) of the statement: “Everyone has exactly one mother”? The meanings of the predicates used are: • 𝑚𝑜𝑡ℎ𝑒𝑟(𝑦, 𝑥): 𝑦 is the mother of 𝑥 • 𝑛𝑜𝑡𝑒𝑞(𝑥, 𝑦): 𝑥 and 𝑦 are not equal", "𝐴= {0, 1, 2, 3, … } is the set of non-negative integers. Let Ϝ be the set of functions from 𝐴 to itself. For any two functions, 𝑓1, 𝑓2 ∈Ϝ, we define (𝑓1⨀𝑓2)(𝑛) = 𝑓1(𝑛) + 𝑓2(𝑛) for every number 𝑛 in 𝐴. Which of the following is/are CORRECT about the mathematical structure (Ϝ, ⨀)?", "Consider the following deterministic finite automaton (DFA) defined over the alphabet, Σ = {𝑎, 𝑏}. Identify which of the following language(s) is/are accepted by the given DFA.", "A disk of size 512M bytes is divided into blocks of 64K bytes. A file is stored in the disk using linked allocation. In linked allocation, each data block reserves 4 bytes to store the pointer to the next data block. The link part of the last data block contains a NULL pointer (also of 4 bytes). Suppose a file of 1M bytes needs to be stored in the disk. Assume, 1K = 210 and 1M = 220. The amount of space in bytes that will be wasted due to internal fragmentation is ______. (Answer in integer)", "Refer to the given 3-address code sequence. This code sequence is split into basic blocks. The number of basic blocks is ________. (Answer in integer)", "A computer has a memory hierarchy consisting of two-level cache (L1 and L2) and a main memory. If the processor needs to access data from memory, it first looks into L1 cache. If the data is not found in L1 cache, it goes to L2 cache. If it fails to get the data from L2 cache, it goes to main memory, where the data is definitely available. Hit rates and access times of various memory units are shown in the figure. The average memory access time in nanoseconds (ns) is ________. (rounded off to two decimal places)", "In optimal page replacement algorithm, information about all future page references is available to the operating system (OS). A modification of the optimal page replacement algorithm is as follows: The OS correctly predicts only up to next 4 page references (including the current page) at the time of allocating a frame to a page. A process accesses the pages in the following order of page numbers: 1, 3, 2, 4, 2, 3, 1, 2, 4, 3, 1, 4. If the system has three memory frames that are initially empty, the number of page faults that will occur during execution of the process is ________ . (Answer in integer)", "Consider the following database tables of a sports league. player(pid,pname,age) coach(cid,cname) team(tid,tname,city,cid) members(pid,tid) An instance of the table and an SQL query are given.", "Suppose a 5-bit message is transmitted from a source to a destination through a noisy channel. The probability that a bit of the message gets flipped during transmission is 0.01. Flipping of each bit is independent of one another. The probability that the message is delivered error-free to the destination is ______ . (rounded off to three decimal places)", "Suppose a message of size 15000 bytes is transmitted from a source to a destination using IPv4 protocol via two routers as shown in the figure. Each router has a defined maximum transmission unit (MTU) as shown in the figure, including IP header. The number of fragments that will be delivered to the destination is ________ . (Answer in integer)", "Consider a probability distribution given by the density function 𝑃(𝑥). 𝑃(𝑥) = {𝐶𝑥2,for 1 ≤𝑥≤40,for 𝑥< 1 or 𝑥> 4 The probability that 𝑥 lies between 2 and 3, i.e., 𝑃(2 ≤𝑥≤3) is __________. (rounded off to three decimal places)", "Consider a finite state machine (FSM) with one input 𝑋 and one output 𝑓, represented by the given state transition table. The minimum number of states required to realize this FSM is ________. (Answer in integer)", "Consider the given sequential circuit designed using D-Flip-flops. The circuit is initialized with some value (initial state). The number of distinct states the circuit will go through before returning back to the initial state is _________ . (Answer in integer)", "The value printed by the given C program is _______ . (Answer in integer)", "Let LIST be a datatype for an implementation of linked list defined as follows: typedef struct list { int data; struct list *next; } LIST; Suppose a program has created two linked lists, L1 and L2, whose contents are given in the figure below (code for creating L1 and L2 is not provided here). L1 contains 9 nodes, and L2 contains 7 nodes. Consider the following C program segment that modifies the list L1. The number of nodes that will be there in L1 after the execution of the code segment is ________ . (Answer in integer)", "The value printed by the given C program is _______ . (Answer in integer)", "The maximum value of 𝑥 such that the edge between the nodes B and C is included in every minimum spanning tree of the given graph is _________ . (answer in integer)", "In a double hashing scheme, ℎ1(𝑘) = 𝑘 mod 11 and ℎ2(𝑘) = 1 + (𝑘 mod 7) are the auxiliary hash functions. The size 𝑚 of the hash table is 11. The hash function for the i-th probe in the open address table is [ℎ1(𝑘) + 𝑖 ℎ2(𝑘)] mod 𝑚. The following keys are inserted in the given order: 63, 50, 25, 79, 67, 24. The slot at which key 24 gets stored is ___________. (Answer in integer)" ]
predict(text)

array(['General Aptitude', 'General Aptitude', 'General Aptitude',
       'General Aptitude', 'General Aptitude', 'General Aptitude',
       'General Aptitude', 'General Aptitude', 'General Aptitude',
       'General Aptitude', 'Computer Organization and Architecture',
       'Programming and Data Structure', 'Programming and Data Structure',
       'Operating System', 'Operating System', 'Computer Networks',
       'Mathematics', 'Programming and Data Structure',
       'Theory of Computation', 'Mathematics',
       'Programming and Data Structure', 'Computer Networks',
       'Mathematics', 'Digital Logic', 'Digital Logic',
       'Programming and Data Structure',
       'Computer Organization and Architecture', 'Theory of Computation',
       'Operating System', 'Mathematics', 'Mathematics', 'Mathematics',
       'Programming and Data Structure', 'Programming and Data Structure',
       'Programming and Data Structure',
       'Computer Organization and Architecture',
       'Comput